In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/My Drive/Face_Mask.zip"

In [2]:
import cv2
import numpy as np
import os
import csv

In [5]:
trainingDataset = []
img_size = 100
path = "/content/drive/My Drive/Pictures/Train"


In [6]:
classNumber = 0
trainingDataset.clear()

for folder in (os.listdir(path)):
  print(classNumber)
  print("Folder Name:",folder)
  # folder = with_mask ,without_mask
  fp = os.path.join(path,folder)
  # joining folder like /content/Face_Mask/Train/with_mask
  for eachImage in os.listdir(fp):
    imagePath = os.path.join(fp,eachImage)
    img = (cv2.imread(imagePath,cv2.IMREAD_GRAYSCALE))/255
    resize=cv2.resize(img,(img_size,img_size))
    trainingDataset.append([resize,classNumber])
  classNumber = classNumber + 1

0
Folder Name: With Mask
1
Folder Name: Without Mask


In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.utils import to_categorical
import pickle
import time

In [8]:
X = []
Y = []
img_size = 100
np.random.shuffle(trainingDataset)
for features, label in trainingDataset:
    X.append(features)
    Y.append(label)
print(Y) 

[1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1]


In [9]:
X = np.array(X).reshape(-1, img_size, img_size, 1)
Y_binary = to_categorical(Y)
print(Y_binary)

[[0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]]


In [10]:
model = Sequential()

model.add(Conv2D(200, (3, 3), input_shape=(100,100,1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(100, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(50))
model.add(Activation('relu'))
 
model.add(Dense(2))
model.add(Activation('softmax'))

In [11]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'],
              )

In [12]:
model.fit(X, Y_binary,
          batch_size = 32,
          epochs=20, validation_split = 0.1)
 
model.save("/content/drive/My Drive/face_mask/Models/{NAME}.model")

Epoch 1/20
1/1 [==============================] - 4s 4s/step - loss: 0.6881 - accuracy: 0.5312 - val_loss: 0.7162 - val_accuracy: 0.2500
Epoch 2/20
1/1 [==============================] - 2s 2s/step - loss: 0.6463 - accuracy: 0.6250 - val_loss: 2.0228 - val_accuracy: 0.2500
Epoch 3/20
1/1 [==============================] - 2s 2s/step - loss: 0.9540 - accuracy: 0.5312 - val_loss: 0.9887 - val_accuracy: 0.7500
Epoch 4/20
1/1 [==============================] - 2s 2s/step - loss: 1.8163 - accuracy: 0.4688 - val_loss: 0.5774 - val_accuracy: 0.7500
Epoch 5/20
1/1 [==============================] - 2s 2s/step - loss: 0.9493 - accuracy: 0.4688 - val_loss: 0.6429 - val_accuracy: 0.7500
Epoch 6/20
1/1 [==============================] - 2s 2s/step - loss: 0.6828 - accuracy: 0.4688 - val_loss: 0.7398 - val_accuracy: 0.2500
Epoch 7/20
1/1 [==============================] - 2s 2s/step - loss: 0.6741 - accuracy: 0.5312 - val_loss: 0.7584 - val_accuracy: 0.2500
Epoch 8/20
1/1 [=========================

In [13]:
def prepare(filepath):
    img_size = 100 
    img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)/255  
    img_resize = cv2.resize(img, (img_size, img_size))  
    return img_resize.reshape(-1, img_size, img_size, 1)

In [14]:
prediction = model.predict(prepare("/content/drive/My Drive/Pictures/Test/With Mask/check.jpg"))
print((prediction))

CATEGORIES = ["with_mask", "without_mask"]

pred_class = CATEGORIES[np.argmax(prediction)]
print(pred_class)


[[0.9408954  0.05910468]]
with_mask
